In [1]:
import pandas as pd
import pymysql
import re,json,os

In [5]:
base = os.environ['BUG_FREE_EUREKA_BASE']
pathTofile = os.path.join(base,'data','exercise06','E-TABM-713_A-MEXP-1551-analytics.tsv')


with open(os.path.join(base,'secrets.json')) as f:
    secrets = json.load(f)

In [6]:
conn = pymysql.connect(**secrets['test_db'])
cursor = conn.cursor()

cursor.execute('create database if not exists biodb')

c:\python35\lib\site-packages\pymysql\cursors.py:166: Warning: (1007, "Can't create database 'biodb'; database exists")
  result = self._query(query)


1

In [7]:
#load data to dataframe
df = pd.read_csv(pathTofile,sep ='\t')
df.head()

,Gene ID,Gene Name,Design Element,g1_g2.p-value,g1_g2.t-statistic,g1_g2.log2foldchange
0,MIMAT0000062,MIMAT0000062,A_25_P00010086,0.702506,0.625362,0.078492
1,MIMAT0000062,MIMAT0000062,A_25_P00011584,0.975514,0.065110,0.007410
2,MIMAT0000063,MIMAT0000063,A_25_P00010070,0.139014,2.431636,0.556112
3,MIMAT0000063,MIMAT0000063,A_25_P00010071,0.049728,3.254492,0.811621
4,MIMAT0000064,MIMAT0000064,A_25_P00010072,0.098283,2.697813,0.757066


In [10]:
#create table from data frame

cursor.execute('use biodb')

#fix column names
df.columns = [re.sub('[ .-]','',x) for x in df.columns]
df.to_sql('dea',conn,flavor='mysql')

c:\python35\lib\site-packages\pandas\core\generic.py:1165: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  chunksize=chunksize, dtype=dtype)


In [11]:
#check if table is created
cursor.execute("show tables")
cursor.fetchall()

(('ache',),
 ('dea',),
 ('drugbank',),
 ('temp',),
 ('temp1',),
 ('temp12',),
 ('uniprot',))

### Exercises

* write a query to fetch the first 10 lines from dea
* Count the number of rows in the table
* Calculat the average p_value for the entire data
* Calculate average p_value for each gene
* Find the gene with highest fold change, lowest fold change
* add your notebook to your GitHub - to your BioDb2016 repository

In [14]:
#Ex 1
cursor.execute("select * from dea limit 10")
cursor.fetchall()

((0,
  'MIMAT0000062',
  'MIMAT0000062',
  'A_25_P00010086',
  0.70250559386657,
  0.625362183554288,
  0.0784915951074403),
 (1,
  'MIMAT0000062',
  'MIMAT0000062',
  'A_25_P00011584',
  0.975513746316429,
  0.0651101242194695,
  0.00741007045087905),
 (2,
  'MIMAT0000063',
  'MIMAT0000063',
  'A_25_P00010070',
  0.139013519150447,
  2.43163612513429,
  0.556111895299825),
 (3,
  'MIMAT0000063',
  'MIMAT0000063',
  'A_25_P00010071',
  0.0497282355402996,
  3.25449181729957,
  0.811621134771325),
 (4,
  'MIMAT0000064',
  'MIMAT0000064',
  'A_25_P00010072',
  0.098282650062664,
  2.69781280670979,
  0.757065824690136),
 (5,
  'MIMAT0000064',
  'MIMAT0000064',
  'A_25_P00010073',
  0.324778784391911,
  1.6485897929498,
  0.440504336234509),
 (6,
  'MIMAT0000065',
  'MIMAT0000065',
  'A_25_P00010281',
  0.103149521552763,
  2.66619309645875,
  0.382716406596201),
 (7,
  'MIMAT0000065',
  'MIMAT0000065',
  'A_25_P00010282',
  0.217890974906888,
  2.04707816939775,
  0.287800723552511),
 (8

In [15]:
#ex2
cursor.execute("select count(*) as num from dea")
cursor.fetchone()

(1230,)

In [16]:
#ex3 - average p_value for entire data
cursor.execute("desc dea")
cursor.fetchall()

(('index', 'bigint(20)', 'YES', 'MUL', None, ''),
 ('GeneID', 'varchar(63)', 'YES', '', None, ''),
 ('GeneName', 'varchar(63)', 'YES', '', None, ''),
 ('DesignElement', 'varchar(63)', 'YES', '', None, ''),
 ('g1_g2pvalue', 'double', 'YES', '', None, ''),
 ('g1_g2tstatistic', 'double', 'YES', '', None, ''),
 ('g1_g2log2foldchange', 'double', 'YES', '', None, ''))

In [17]:
cursor.execute("select avg(g1_g2pvalue) from dea")
cursor.fetchall()

((0.4529776486719517,),)

In [18]:
#ex4- avg p_value for each gene
cursor.execute("""select GeneID,avg(g1_g2pvalue) 
from dea group by GeneID""")
cursor.fetchall()

(('MIMAT0000062', 0.8390096700914995),
 ('MIMAT0000063', 0.09437087734537329),
 ('MIMAT0000064', 0.2115307172272875),
 ('MIMAT0000065', 0.1605202482298255),
 ('MIMAT0000066', 0.421072681151927),
 ('MIMAT0000067', 0.3668396148775826),
 ('MIMAT0000068', 0.172509684242204),
 ('MIMAT0000069', 0.040891551036128196),
 ('MIMAT0000070', 0.4276906754653405),
 ('MIMAT0000071', 0.891390789294826),
 ('MIMAT0000072', 0.32429179301055),
 ('MIMAT0000073', 0.5245145043568785),
 ('MIMAT0000074', 0.290672380835091),
 ('MIMAT0000075', 0.464782884457078),
 ('MIMAT0000076', 0.4530537439696015),
 ('MIMAT0000077', 0.641912313397985),
 ('MIMAT0000078', 0.0212501986719539),
 ('MIMAT0000079', 0.718276941344944),
 ('MIMAT0000080', 0.603024955133693),
 ('MIMAT0000081', 0.2336861207054201),
 ('MIMAT0000082', 0.0031096021726969205),
 ('MIMAT0000083', 0.05413238635436655),
 ('MIMAT0000084', 0.8315172458771201),
 ('MIMAT0000085', 0.32132354018986153),
 ('MIMAT0000086', 0.1032058916642123),
 ('MIMAT0000087', 0.8714517

In [19]:
#ex5- gene with highest fold change, lowest fold change
cursor.execute("""select GeneID
                from dea
                where g1_g2log2foldchange = 
                (select min(g1_g2log2foldchange) from dea)""")

cursor.execute("""select GeneID
                from dea
                where g1_g2log2foldchange = 
                (select max(g1_g2log2foldchange) from dea)""")

cursor.execute("""select GeneID
                from dea
                where g1_g2log2foldchange = 
                (select avg(g1_g2log2foldchange) from dea)""")
cursor.fetchall()

(('MIMAT0000451',),)

In [22]:
#Wrong solution - we do not know if the gene id is for the minimum value or the maximum value

cursor.execute("""select GeneID,
                min(g1_g2log2foldchange),
                max(g1_g2log2foldchange)
                from dea""")
cursor.fetchall()

(('MIMAT0000062', -1.44215632547793, 2.68544495738703),)